# ETL- Car Price Analysis
US Market (Kaggle: hellbuoy/car-price-prediction)

## Objectives

Load and clean the car price dataset to prepare it for analysis.  
This includes standardising column names, correcting brand typos, handling missing values, engineering useful features, encoding categorical variables, and performing basic validation checks.  
The cleaned dataset will be ready for Exploratory Data Analysis (EDA) and documentation in the README.

## Inputs

 **Primary dataset**: `data/CarPrice_Assignment.csv` — contains information on car specifications and pricing in the US market.
- **Data fields**: Includes `CarName`, `fueltype`, `aspiration`, `doornumber`, `carbody`, `drivewheel`, `enginelocation`, `enginesize`, `horsepower`, `citympg`, `highwaympg`, `price`, and other technical attributes.
- **Tools & libraries**: Python 3.x with `pandas`, `numpy`, and `matplotlib` installed.

## Outputs

- Cleaned DataFrame (`df_enc`) with standardised columns, corrected brand names, engineered features, and one-hot encoded categoricals.
- Optional cleaned CSV saved as `data/car_price_clean.csv`.
- Basic validation checks confirming data integrity.




---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

Section 1 content

# Section 1 - Extract & Inspect
Load the CSV, confirm shape, peek at columns & missing values.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
import pandas as pd

# Correct path to the CSV file inside the 'data' folder
file_path = 'data/CarPrice_Assignment.csv'

# Check if file exists in the 'data' directory
if not os.path.isfile(file_path):
    print("❌ File not found. Files in 'data' folder:")
    print(os.listdir('data'))  # Optional: show what's inside 'data' folder
else:
    df = pd.read_csv(file_path)
    df.head()

In [ ]:
from pathlib import Path
import pandas as pd

DATA_PATH = Path("data")
IMG_PATH  = Path("images")

DATA_PATH.mkdir(parents=True, exist_ok=True)
IMG_PATH.mkdir(parents=True, exist_ok=True)

input_file = DATA_PATH / "CarPrice_Assignment.csv"
assert input_file.exists(), f"Missing file: {input_file}. Place it under data/."

df = pd.read_csv(input_file)
print("Shape:", df.shape)
df.head()

In [ ]:
# Column info & dtypes
df.info()

In [ ]:
# Missing values snapshot
df.isna().sum().sort_values(ascending=False).head(25)

---

# Section 2- Transform
Steps:
- Standardise column names
- Split `CarName` → `make` / `model`
- Basic NA handling on key fields
- Clean brand typos (common in this dataset)
- Feature engineering (e.g., price_per_cc, power_to_weight, mpg_ratio)
- One-hot encode selected categoricals
- Light outlier clipping for price (optional)

In [ ]:
import numpy as np

# 1) Standardise column names: lowercase + underscores
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("-", "_")
)

# 2) Split `carname` to `make` / `model` if present
if "carname" in df.columns:
    split_make_model = df["carname"].str.split(" ", n=1, expand=True)
    df["make"]  = split_make_model[0].str.lower()
    df["model"] = split_make_model[1].str.lower()
else:
    print("Column 'carname' not found; skipping make/model split.")

# 3) Ensure critical fields exist, then handle NAs
critical_cols = ["price", "enginesize"]
for c in critical_cols:
    if c not in df.columns:
        raise KeyError(f"Expected column '{c}' not found in dataset.")

before_rows = len(df)
df = df.dropna(subset=critical_cols)
after_rows = len(df)
print(f"Dropped {before_rows - after_rows} rows with missing critical values.")

# 4) Clean common brand typos in 'make'
brand_fix = {
    "maxda": "mazda",
    "toyouta": "toyota",
    "vokswagen": "volkswagen",
    "vw": "volkswagen",
    "porcshe": "porsche",
}
if "make" in df.columns:
    df["make"] = df["make"].replace(brand_fix)

# 5) Feature engineering
# - price_per_cc: price per engine cc
# - power_to_weight: horsepower per curbweight
# - mpg_ratio: highway / city mpg (efficiency balance)
df["price_per_cc"] = df["price"] / df["enginesize"]

if {"horsepower", "curbweight"}.issubset(df.columns):
    df["power_to_weight"] = df["horsepower"] / df["curbweight"].replace(0, np.nan)

if {"highwaympg", "citympg"}.issubset(df.columns):
    df["mpg_ratio"] = df["highwaympg"] / df["citympg"].replace(0, np.nan)

# 6) Optional: clip extreme price outliers (1st–99th percentile)
low, high = df["price"].quantile([0.01, 0.99])
df["price_clipped"] = df["price"].clip(lower=low, upper=high)

# 7) One-hot encode selected categoricals
categoricals = [c for c in [
    "fueltype", "aspiration", "doornumber", "carbody",
    "drivewheel", "enginelocation", "enginetype",
    "cylindernumber", "fuelsystem", "make"
] if c in df.columns]

df_enc = pd.get_dummies(df, columns=categoricals, drop_first=True)

print("Encoded shape:", df_enc.shape)
df_enc.head()

## Validate
Sanity checks: no negative prices, positive engine sizes, and basic NA review post-transform.

In [ ]:
# Price should be non-negative
assert df_enc["price"].ge(0).all(), "Found negative prices."

# Enginesize should be positive
assert df_enc["enginesize"].gt(0).all(), "Found non-positive enginesize."

# Quick NA check after transforms/encodings
na_counts = df_enc.isna().sum().sum()
print(f"Total remaining NA values across all columns: {na_counts}")

### Final NA Cleanup

After all transformations, we found a small number of missing values remaining in the processed dataset.  
For a fully clean dataset, we will remove any rows containing NA values.  
Alternatively, you can fill missing values with a default (e.g., 0 or the column mean), depending on your analysis needs.

In [ ]:
# Drop any rows with missing values in the processed DataFrame
df_enc = df_enc.dropna()
print(f"Total remaining NA values across all columns: {df_enc.isna().sum().sum()}")

In [ ]:
# Fill any remaining missing values with 0
df_enc = df_enc.fillna(0)
print(f"Total remaining NA values across all columns: {df_enc.isna().sum().sum()}")

## Section 3- Load and Save Cleaned Data
If you’d like a cleaned copy for modelling/visuals, save it back into the **same `data/` folder**.

In [ ]:
output_file = DATA_PATH / "car_price_clean.csv"
df_enc.to_csv(output_file, index=False)
print(f"Saved processed dataset to: {output_file}")

## Conclusion (ETL)

- The dataset was successfully loaded from `data/CarPrice_Assignment.csv`.
- Columns were standardised (lowercase, underscores), and `CarName` was split into `make` and `model`.
- Critical fields (`price`, `enginesize`) were validated and rows with missing critical values were removed.
- Common brand typos (e.g., `maxda`, `toyouta`, `vokswagen`, `porcshe`) were corrected.
- Key engineered features were added:
  - `price_per_cc` – price normalised by engine size
  - `power_to_weight` – horsepower per curb weight (when available)
  - `mpg_ratio` – highway-to-city MPG ratio (when available)
- Optional outlier clipping (1st–99th percentile) was applied to `price` for robust downstream analysis.
- Selected categorical variables were one-hot encoded, producing a clean numeric table (`df_enc`) appropriate for visualisation and modelling.
- Basic data-quality checks passed (non-negative prices, positive engine sizes; NA counts reviewed).

## Next Steps

1. **Exploratory Data Analysis (EDA)**
   - Perform in-depth visual exploration of the dataset using both static and interactive charts.
   - Identify key relationships between features and price.
   - Detect additional data anomalies or unusual patterns.

2. **README Documentation**
   - Update the project README to include:
     - Project background and business context.
     - ETL process summary.
     - Key dataset insights.
     - Instructions to reproduce the ETL process.
   - Include images generated during ETL and EDA for better communication.